In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, Embedding, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import os

# Load the dataset /kaggle/input/fashion-product-text-images-dataset
DATA_PATH = "../input/fashion-product-text-images-dataset"
CSV_FILE = os.path.join(DATA_PATH, "data.csv")
IMAGE_PATH = os.path.join(DATA_PATH, "data")

data = pd.read_csv(CSV_FILE)

# Preview the dataset
data.dropna(subset=["image", "description", "category"], inplace=True)
print("Dataset Preview:")
print(data.head())

# Encode the categories
label_encoder = LabelEncoder()
data['category_encoded'] = label_encoder.fit_transform(data['category'])

# Split the data
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Preprocessing for images
def preprocess_image(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    return img

def load_images(df):
    images = []
    for img_name in df['image']:
        img_path = os.path.join(IMAGE_PATH, img_name)
        images.append(preprocess_image(img_path))
    return np.array(images)

train_images = load_images(train_data)
val_images = load_images(val_data)

# Preprocessing for text
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LEN = 100
MAX_WORDS = 10000
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(train_data['description'])

train_text = tokenizer.texts_to_sequences(train_data['description'])
val_text = tokenizer.texts_to_sequences(val_data['description'])

train_text = pad_sequences(train_text, maxlen=MAX_LEN)
val_text = pad_sequences(val_text, maxlen=MAX_LEN)

# Normalize the target
scaler = StandardScaler()
train_labels = tf.keras.utils.to_categorical(train_data['category_encoded'])
val_labels = tf.keras.utils.to_categorical(val_data['category_encoded'])

# Build the image model
image_input = Input(shape=(224, 224, 3), name="image_input")
base_model = EfficientNetB0(include_top=False, input_tensor=image_input)
image_features = GlobalAveragePooling2D()(base_model.output)
image_features = Dropout(0.3)(image_features)

# Build the text model
text_input = Input(shape=(MAX_LEN,), name="text_input")
embedding = Embedding(input_dim=MAX_WORDS, output_dim=128, input_length=MAX_LEN)(text_input)
text_features = tf.keras.layers.LSTM(128)(embedding)
text_features = Dropout(0.3)(text_features)

# Multimodal fusion
merged = Concatenate()([image_features, text_features])
output = Dense(len(label_encoder.classes_), activation="softmax", name="output")(merged)

# Define the model
model = Model(inputs=[image_input, text_input], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

# Model summary
model.summary()

# Train the model
history = model.fit(
    [train_images, train_text], train_labels,
    validation_data=([val_images, val_text], val_labels),
    epochs=10,
    batch_size=32
)

# Plot results
def plot_history(history):
    plt.figure(figsize=(12, 4))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.legend()

    plt.show()

plot_history(history)

# Save the model
model.save("fashion_recommendation_model.h5")
